In [2]:
import numpy as np
from astropy.table import Table
import matplotlib.pyplot as plt
from desitarget.targetmask import desi_mask, bgs_mask
from collections import Counter

In [3]:
targets = Table.read('targets/cut_dark_north.fits')
tiles = Table.read('footprint/tiles_cut_dark_north.fits')

In [21]:
isELG = (targets['DESI_TARGET']&desi_mask['ELG'])!=0
print(np.count_nonzero(isELG), len(targets))
print(np.count_nonzero(targets['OBSCONDITIONS'][isELG]==3))
print(set(tiles['OBSCONDITIONS']))
gray_obsconditions = 2
dark_obsconditions = 1
dark_or_gray_obsconditions = gray_obsconditions | dark_obsconditions

2038825 3902588
2037983
{1, 2}


In [20]:
gray_obsconditions|dark_obsconditions

3

In [22]:
r = np.random.random(len(targets))
rr = r<0.30

In [23]:
to_gray = isELG & rr
to_dark = isELG & (~rr)
targets['OBSCONDITIONS'][to_gray] = gray_obsconditions
targets['OBSCONDITIONS'][to_dark] = dark_obsconditions

In [24]:
targets.write('targets/cut_dark_north_elg_30_gray_70_dark.fits')